In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.signal as signal
import scipy.stats as stats
import plotly.io as pio
from IPython.display import display, HTML
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.filters.cf_filter import cffilter
from statsmodels.tsa.stattools import adfuller

pio.renderers.default = "notebook_connected"

def show_plotly(fig, include_plotlyjs="inline"):
    """Helper do wyświetlania wykresów Plotly."""
    try:
        fig.show()
    except Exception:
        html = fig.to_html(full_html=False, include_plotlyjs='cdn')
        display(HTML(html))


Teraz tworzę funkcje pomocnicze, które pozwolą mi na łatwe wyznaczanie stóp zwrotu (prostą i logarytmiczną)
W dalszej części projektu skupię się na analizie **stóp logarytmicznych**.

In [2]:
def compute_returns(series: pd.Series) -> pd.DataFrame:
    returns = pd.DataFrame(index=series.index)
    returns["simple_frac"] = (series - series.shift(1)) / series.shift(1)
    returns["log_frac"] = np.log(series / series.shift(1))
    return returns

def compute_dft_complex(x):
    """
    Oblicza DFT (Dyskretną Transformatę Fouriera) przy użyciu algorytmu FFT.
    Zwraca współczynniki X_k (liczby zespolone).
    """
    return np.fft.fft(x)

def compute_naive_periodogram(x):
    """
    Oblicza periodogram naiwny na podstawie DFT.
    Zwraca wektor częstotliwości (f) i mocy (P).
    """
    N = len(x)
    X_k = compute_dft_complex(x)
    power = (1/N) * (np.abs(X_k)**2)
    freqs = np.fft.fftfreq(N, d=1)
    idx = np.where(freqs >= 0)
    return freqs[idx], power[idx]

In [3]:
data_btcusdt_frame_daily: pd.DataFrame = pd.read_csv(
    Path("./btcusd_day.csv"),
    usecols=["datetime", "open", "high", "low", "close"],
    parse_dates=["datetime"],
)
# Ustawiam odpowiednio indeks
data_btcusdt_frame_daily.set_index("datetime").sort_index().astype("float64")

# konwersja kolumny 'Data' na datetime, bo bez tego są błedne daty
data_btcusdt_frame_daily["datetime"] = pd.to_datetime(
    data_btcusdt_frame_daily["datetime"], errors="coerce"
)

# usuwam ewentualne błędne wiersze bez dat
data_btcusdt_frame_daily = data_btcusdt_frame_daily.dropna(subset=["datetime"])

# indeks i sortuje
data_btcusdt_frame_daily = (
    data_btcusdt_frame_daily.set_index("datetime").sort_index().astype("float64")
)

print(data_btcusdt_frame_daily.head(10))

               open     high      low    close
datetime                                      
2010-07-17  0.04951  0.04951  0.04951  0.04951
2010-07-18  0.08584  0.08584  0.08584  0.08584
2010-07-19  0.08080  0.08080  0.08080  0.08080
2010-07-20  0.07474  0.07474  0.07474  0.07474
2010-07-21  0.07921  0.07921  0.07921  0.07921
2010-07-22  0.05050  0.05050  0.05050  0.05050
2010-07-23  0.06262  0.06262  0.06262  0.06262
2010-07-24  0.05454  0.05454  0.05454  0.05454
2010-07-25  0.05050  0.05050  0.05050  0.05050
2010-07-26  0.05600  0.05600  0.05600  0.05600


In [4]:
strategy_equity_frame_daily: pd.DataFrame = pd.read_csv(
    Path("./equity_daily_normalized.csv"),
    usecols=["datetime", "equity"],
    parse_dates=["datetime"],
)
# strategy_equity_frame_daily: pd.DataFrame = pd.read_csv(
#     Path("./equity_daily_not_normalized.csv"),
#     usecols=["datetime", "equity"],
#     parse_dates=["datetime"],
# )

# Ustawiam odpowiednio indeks
strategy_equity_frame_daily.set_index("datetime").sort_index().astype("float64")

# konwersja kolumny 'Data' na datetime, bo bez tego są błedne daty
strategy_equity_frame_daily["datetime"] = pd.to_datetime(
    strategy_equity_frame_daily["datetime"], errors="coerce"
)

# usuwam ewentualne błędne wiersze bez dat
strategy_equity_frame_daily.dropna(subset=["datetime"])

# indeks i sortuje
strategy_equity_frame_daily = (
    strategy_equity_frame_daily.set_index("datetime").sort_index().astype("float64")
)

strategy_equity_frame_daily.head(10)

,equity
datetime,
2022-01-01 00:00:00+00:00,1.000000
2022-01-02 00:00:00+00:00,1.000000
2022-01-03 00:00:00+00:00,1.000000
2022-01-04 00:00:00+00:00,0.997370
2022-01-05 00:00:00+00:00,1.050057
2022-01-06 00:00:00+00:00,1.057094
2022-01-07 00:00:00+00:00,1.090937
2022-01-08 00:00:00+00:00,1.088179
2022-01-09 00:00:00+00:00,1.084254


In [5]:
# Wybór danych
# 1. Wybór danych (zgodnie z wyborem w GUI/kodu powyżej)
# raw_series = data_btcusdt_frame_daily['close'].values
raw_series = compute_returns(data_btcusdt_frame_daily['close'])["log_frac"]
# raw_series = strategy_equity_frame_daily['equity'].values

# 0 Przygotowanie danych i sprawdzenie stacjonarności

W materiałach $\lambda$ często jest ustawiona na 1600 dla danych kwartalnych. Natomiast dla dziennych często używa się 14400 lub 100000.

W literaturze (tzw. reguła Ravna-Uhliga), parametr $\lambda$ skaluje się z czwartą potęgą częstotliwości, co by nam dawało:

- kwartalnie: $1600$
- miesięczne: $1600 \times 3^4 = 129\ 600$ (często upraszczane do 144 000)
- dziennie (ok. 90 dni w kwartale): $1600 \times 90^4 \approx \mathbf{100\ 000\ 000} \ (10^8)$

Profesjonalnie dla danych dziennych powinniśmy więc użyć $\lambda=\mathbf{100\ 000\ 000} \ (10^8)$, aby trend był "gładki", a cykle wyraźne.
$10^8$ wygląda na ogromną wartość parametru, dlatego ja ustawiłem $\lambda=\mathbf{6 \ 400 \ 000}$ (standard dla wielu analiz finansowych).

Filtr CF
Pominać logarytmowwnie
dodać jednostkę czasu

In [6]:
# 1. Filtr Hodricka-Prescotta (HP)
print(f"Przetwarzanie serii HP: {len(raw_series)} punktów")

data_series_hp = raw_series.dropna()
hp_lambda = 6_400_000
cycle_hp, trend_hp = hpfilter(data_series_hp, lamb=hp_lambda)

def plot_hp_results(raw, trend, cycle):
    fig = make_subplots(rows=3, cols=1, 
                         subplot_titles=("1. Surowe log-zwroty (Dane wejściowe)", 
                                         "2. Wyizolowany trend (Low-pass component)", 
                                         "3. Składowa cykliczna (Stacjonarna - wysokoczęstotliwościowa)"))
    
    fig.add_trace(go.Scatter(y=raw, name="Raw"), row=1, col=1)
    fig.add_trace(go.Scatter(y=trend, name="Trend"), row=2, col=1)
    fig.add_trace(go.Scatter(y=cycle, name="Cycle"), row=3, col=1)
    
    fig.update_layout(title_text="Filtr Hodricka-Prescotta (HP)", height=900, showlegend=False)
    show_plotly(fig)

plot_hp_results(data_series_hp, trend_hp, cycle_hp)

Przetwarzanie serii HP: 5607 punktów


### Dlaczego wykresy wyglądają tak, a nie inaczej?

#### 1. HP: Podobieństwo "Raw" vs "Cycle"
Filtr Hodricka-Prescotta jest filtrem typu górnoprzepustowego (high-pass). Ponieważ analizujemy teraz **log-zwroty** (które same w sobie są już wynikiem filtrowania cen i są stacjonarne), filtr HP nie ma w nich "czego wycinać" poza bardzo długoterminowym trendem. Jeśli $\lambda$ jest wysokie (np. 6.4M), trend jest prawie płaską linią przy zerze. Dlatego `cykl = dane - trend` wygląda prawie identycznie jak dane wejściowe. To dowodzi, że zwroty są już "wysokoczęstotliwościowe".

#### 2. CF: Problem stacjonarności i brzegów
- **Efekty brzegowe**: Na samym początku wykresu CF widać gwałtowny spadek (do ok. -0.2). Jest to typowy artefakt filtrów pasmowoprzepustowych przy krawędziach danych (tzw. end effects). Filtr potrzebuje "okna" danych, by poprawnie działać.
- **Parametry [500, 1500]**: Przy tak długich cyklach (halving to ok. 1460 dni), ADF może zgłaszać niestacjonarność, jeśli okno jest zbyt szerokie, bo sygnał staje się zbyt gładki i "leniwy". Zmieniliśmy zakres na 500-1500 dni, co daje p-value < 0.05, czyniąc szereg stacjonarnym w sensie statystycznym, przy jednoczesnym zachowaniu kluczowych cykli.

In [7]:
# 2. Filtr Christiano-Fitzgeralda (CF)
# Filtr pasmowoprzepustowy - isolujemy cykle [100, 1500] dni
print(f"Przetwarzanie serii CF: {len(raw_series)} punktów")

data_series_cf = raw_series.dropna()
# low=100, high=1500, drift=False (dla zwrotów lepiej bez driftu)
cycle_cf, trend_cf = cffilter(data_series_cf, low=100, high=1500, drift=False)

def plot_cf_results(raw, trend, cycle):
    fig = make_subplots(rows=3, cols=1, 
                         subplot_titles=("1. Surowe log-zwroty (Dane wejściowe)", 
                                         "2. Wyizolowany trend (Slow-moving components)", 
                                         "3. Składowa cykliczna (Cykle [100, 1500] dni)"))
    
    fig.add_trace(go.Scatter(y=raw, name="Raw"), row=1, col=1)
    fig.add_trace(go.Scatter(y=trend, name="Trend"), row=2, col=1)
    fig.add_trace(go.Scatter(y=cycle, name="Cycle (Band-passed)"), row=3, col=1)
    
    fig.update_layout(title_text="Filtr Christiano-Fitzgeralda (CF)", height=1000, showlegend=False)
    show_plotly(fig)

plot_cf_results(data_series_cf, trend_cf, cycle_cf)

Przetwarzanie serii CF: 5607 punktów


### Sprawdzenie stacjonarności danych

Zanim przejdziemy do punktu 1. projektu, sprawdzimy czy dane po filtracji są stacjonarne.

In [8]:
# Weryfikacja stacjonarności (Test ADF) ---
def check_stationarity(timeseries, name: str):
    print(f"-" * 60)
    print(f"Test stacjonarności (ADF) dla: {name}")
    
    # Usuwamy NaN jeśli są
    ts = timeseries.dropna() if hasattr(timeseries, 'dropna') else timeseries

    result = adfuller(ts, autolag="AIC")
    p_value = result[1]
    print(f"Statystyka ADF: {result[0]:.4f}")
    print(f"p-value: {p_value:.10f}")

    if p_value <= 0.05:
        print("p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.")
    else:
        print("p-value > 0.05. Nie ma podstaw do odrzucenia H0. Szereg jest NIESTACJONARNY.")

check_stationarity(raw_series, "Log-zwroty (Raw)")
check_stationarity(cycle_hp, "Składowa cykliczna HP")
check_stationarity(cycle_cf, "Składowa cykliczna CF")

------------------------------------------------------------
Test stacjonarności (ADF) dla: Log-zwroty (Raw)
Statystyka ADF: -12.8864
p-value: 0.0000000000
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.
------------------------------------------------------------
Test stacjonarności (ADF) dla: Składowa cykliczna HP
Statystyka ADF: -27.4483
p-value: 0.0000000000
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.
------------------------------------------------------------
Test stacjonarności (ADF) dla: Składowa cykliczna CF
Statystyka ADF: -3.6323
p-value: 0.0051719292
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.


Zastosowanie testu Augmented Dickey-Fuller (ADF) potwierdziło stacjonarność składowej cyklicznej uzyskanej po filtracji HP ($p-value \approx 0.0$). Wysoka wartość parametru lambda ($6.4 \times 10^6$) pozwoliła na precyzyjne odizolowanie długookresowego trendu przy jednoczesnym zachowaniu istotnych informacji o cyklach rynkowych. Periodogram naiwny wykazuje silną koncentrację mocy w zakresie niskich częstotliwości ($f < 0.05$), co sugeruje dominację cykli trwających powyżej 20 dni. Ze względu na dużą wariancję estymatora naiwnego (charakterystyczny 'las szpilek'), w kolejnym kroku konieczne jest zastosowanie wygładzania spektralnego.

In [9]:
# Obliczamy periodogramy dla wszystkich składowych
f_ret, p_ret = compute_naive_periodogram(raw_series.dropna())
f_hp, p_hp = compute_naive_periodogram(cycle_hp)
f_cf, p_cf = compute_naive_periodogram(cycle_cf)

# Tworzymy wykres porównawczy
fig = go.Figure()

fig.add_trace(go.Scatter(x=f_ret[1:], y=p_ret[1:], name="Log-zwroty (Raw)", line=dict(color='gray', width=1)))
fig.add_trace(go.Scatter(x=f_hp[1:], y=p_hp[1:], name="Cykl HP", line=dict(color='blue', width=1.5)))
fig.add_trace(go.Scatter(x=f_cf[1:], y=p_cf[1:], name="Cykl CF", line=dict(color='orange', width=2)))

fig.update_layout(
    title="Porównanie periodogramów (Zoom na niskie częstotliwości)",
    xaxis_title="Częstotliwość (f)",
    yaxis_title="Moc (Power)",
    yaxis_type="log",
    xaxis_range=[0, 0.05], # ZOOM: skupiamy się na cyklach > 20 dni (1/0.05)
    hovermode="x unified"
)
show_plotly(fig)

### Wyjaśnienie "dziwnego" wyglądu filtra CF:

1. **Dlaczego periodogram CF to gładka linia?**
To nie błąd, to **dowód działania filtra**. Filtr CF jest filtrem pasmowoprzepustowym (band-pass). Ustawiliśmy go na zakres 100-1500 dni ($f \in [0.00067, 0.01]$). 
- To, co widzisz jako "wahania" na początku wykresu (lewa strona), to właśnie Twoje wyizolowane cykle.
- Wszystko na prawo od $f = 0.01$ (czyli 98% wykresu) zostało przez filtr **wycięte do zera**. Gładka linia schodząca w dół na skali logarytmicznej to po prostu charakterystyka wygaszania filtra. Na skali liniowej ta linia byłaby płaska przy samym zerze.

2. **Dlaczego 100-1500 dni jest niestacjonarne?**
Przy `low=100`, filtr dopuszcza zbyt dużo szybkich zmian, które przy silnym halvingu i specyfice Bitcoina mogą dawać szereg, który ADF uznaje za niestacjonarny (pamiętajmy, że ADF szuka pierwiastka jednostkowego). Ustawienie `low=500` (lub więcej) mocniej wygładza szereg, czyniąc go statystycznie stacjonarnym. W przypadku finansów, analiza cykli 100-dniowych jest jednak bardzo ciekawa, więc zostawiamy ją, akceptując fakt, że szereg jest "na granicy" stacjonarności.

3. **Drift=False**: Wyłączenie driftu sprawia, że składowa trendu nie próbuje na siłę dopasować się do wolnych zmian poziomu zwrotów, co daje stabilniejszą bazę cykliczną.

## Eksperyment: Podzielność liczby obserwacji N przez naturalny okres T

W tym punkcie badamy zjawisko **przecieku widma (spectral leakage)**. Teoria sugeruje, że jeśli liczba obserwacji $N$ nie jest wielokrotnością naturalnego okresu $T$ występującego w danych, energia sygnału "rozlewa się" na sąsiednie częstotliwości.

W danych finansowych za naturalne okresy możemy przyjąć:
* $T = 7$ (okres tygodniowy),
* $T = 365$ (okres roczny).

In [10]:
# Konfiguracja eksperymentu
natural_period = 365  # Cykl roczny
max_p = None        # None = bierze wszystkie

def run_divisibility_experiment(series, name):
    N_total = len(series)
    N_divisible = (N_total // natural_period) * natural_period
    N_non_divisible = N_divisible + (natural_period // 2)

    # Przycinanie danych
    x_div = series.iloc[:N_divisible]
    x_non = series.iloc[:N_non_divisible]

    f_div, p_div = compute_naive_periodogram(x_div)
    f_non, p_non = compute_naive_periodogram(x_non)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=f_div[1:], y=p_div[1:], name=f"N podzielne przez T={natural_period} (N={N_divisible})"))
    fig.add_trace(go.Scatter(x=f_non[1:], y=p_non[1:], name=f"N niepodzielne (N={N_non_divisible})", opacity=0.7))

    fig.update_layout(
        title=f"Eksperyment: Wpływ podzielności N na leakage ({name})",
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc",
        yaxis_type="log",
        xaxis_range=[0, 0.05]
    )
    show_plotly(fig)

run_divisibility_experiment(cycle_hp, "Cykl HP")
run_divisibility_experiment(cycle_cf, "Cykl CF")

### Komentarz do eksperymentu i wnioski:

Przeprowadzony eksperyment porównujący periodogram dla liczby obserwacji $N$ podzielnej przez $T=7$ (lub $T=365$) oraz niepodzielnej, prowadzi do następujących wniosków:

1. **Brak wyraźnych różnic wizualnych**: Na powyższym wykresie obie linie (zielona i czerwona) niemal całkowicie się pokrywają (w oddali bez przybliżenia). Efekt "przecieku widma" (spectral leakage), który w teorii powinien powodować rozmycie pików przy $N$ niepodzielnym, jest w tym przypadku niewidoczny dla oka.
2. **Przyczyny braku widoczności efektu**:
   * **Wysoka wariancja periodogramu naiwnego**: Dominującą cechą periodogramu naiwnego jest to, że jest on estymatorem niezgodnym, jego wykres to ogromna ilość pojedynczych pików. Ta naturalna zmienność (szum) całkowicie przykrywa drobne, deterministyczne zniekształcenia wynikające z braku podzielności $N$ przez $T$.
   * **Niedeterministyczny charakter danych**: Bitcoin to realny szereg ekonomiczny, a nie czysta sinusoida. Cykle w takich danych nie są idealnie stałe w czasie, co powoduje naturalne "rozmycie" energii w widmie, niezależnie od doboru $N$.
   * **Duża liczba obserwacji ($N > 5000$)**: Przy tak dużej próbie, rozdzielczość częstotliwościowa jest bardzo wysoka. Błędy wynikające z niedopasowania okna do okresu są rozłożone na bardzo wąskie pasma, co czyni je nieistotnymi w skali całego widma.

## Podsumowanie Punktu 1

Analiza samym periodogramem naiwnym ( DFT ) jest niewystarczająca do precyzyjnej identyfikacji cykli. Duża wariancja estymatora maskuje zarówno subtelne zjawiska teoretyczne (jak leakage), jak i faktyczne cykle koniunkturalne. Stanowi to bezpośrednie uzasadnienie dla przejścia do **Fazy 2: Porównanie metod**, która ma na celu redukcję tej wariancji u wygładzenia periodogramu.

# 2. Porównanie metod

W tej części projektu przechodzimy do bardziej zaawansowanych metod estymacji gęstości widmowej mocy. Naszym celem jest redukcja wariancji periodogramu naiwnego, który jak pokazaliśmy wcześniej jest estymatorem niezgodnym.

## Periodogram wprost z definicji z przedziałem ufności 95%

Zgodnie z teorią, dla dużych $N$, statystyka $2I(f)/S(f)$ ma rozkład chi-kwadrat z 2 stopniami swobody ($\chi^2_2$). Pozwala to na wyznaczenie przedziału ufności dla prawdziwej gęstości widmowej $S(f)$ na podstawie wyznaczonego periodogramu $I(f)$:

$$ \frac{2 I(f)}{\chi^2_{1-\alpha/2}(2)} \leq S(f) \leq \frac{2 I(f)}{\chi^2_{\alpha/2}(2)} $$

Dla poziomu ufności 95% ($\alpha = 0.05$), wyznaczamy kwantyle rozkładu $\chi^2$ dla $0.975$ oraz $0.025$.


In [11]:
from scipy.stats import chi2

def plot_periodogram_with_ci(freqs, power, alpha=0.05, title="Periodogram with 95% CI"):
    N = len(freqs)
    # Dla p-gramu naiwnego: 2 * I(f) / chi2(2)
    lower_chi = chi2.ppf(1 - alpha/2, df=2)
    upper_chi = chi2.ppf(alpha/2, df=2)
    
    ci_lower = (2 * power) / lower_chi
    ci_upper = (2 * power) / upper_chi
    
    fig = go.Figure()
    
    # Obszar ufności (shading)
    fig.add_trace(go.Scatter(
        x=np.concatenate([freqs[1:], freqs[1:][::-1]]),
        y=np.concatenate([ci_upper[1:], ci_lower[1:][::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        name="95% CI"
    ))
    
    fig.add_trace(go.Scatter(x=freqs[1:], y=power[1:], name="Periodogram", line=dict(color='rgb(0,100,80)')))
    
    fig.update_layout(
        title=title,
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc (Log scale)",
        yaxis_type="log",
        xaxis_range=[0, 0.05],
        showlegend=True
    )
    show_plotly(fig)

# Osobne wykresy dla HP i CF
plot_periodogram_with_ci(f_hp, p_hp, title="Periodogram Składowej Cyklicznej HP z 95% CI")
plot_periodogram_with_ci(f_cf, p_cf, title="Periodogram Składowej Cyklicznej CF z 95% CI")

### Wnioski i Interpretacja:

1. **Szerokość przedziału ufności**: Zastosowanie skali logarytmicznej na osi Y pozwala zauważyć, że przedział ufności ma stałą szerokość w sensie proporcjonalnym. Oznacza to, że błąd względny estymacji jest taki sam dla każdej częstotliwości. Bandy dolna i górna są tak samo oddalone od wartości Y.
2. **Niezgodność estymatora**: Przedział ufności jest bardzo szeroki. Fakt, że nie zwęża się on wraz ze wzrostem $N$ (co wykazaliśmy teoretycznie), potwierdza, że periodogram naiwny nie zbiega do prawdziwej wartości gęstości widmowej.
3. **Piki istotne statystycznie**: Dopiero pik, który wyraźnie "wystaje" ponad nasze tło i szum oraz którego przedział ufności nie pokrywa się z szerokim pasmem szumu, może być uznany za potencjalny cykl (wyróżniający się pik na przestrzeni danych). W przypadku periodogramu naiwnego większość szpilek jest wynikiem losowości, a nie faktycznych cykli, co widać po tym, jak gęsto szpile przeskakują między górną a dolną granicą oraz jak bardzo są podobne.

## Wygładzanie spektralne przy użyciu okna Daniella

Jednym z najprostszych sposobów na otrzymanie zgodnego estymatora gęstości widmowej jest wygładzanie periodogramu naiwnego w dziedzinie częstotliwości. Metoda ta polega na uśrednianiu wartości periodogramu w otoczeniu danej częstotliwości (średnia ruchoma).

Estymator z oknem Daniella o szerokości $L = 2m + 1$ dany jest wzorem:
$$ \hat{S}(f_k) = \frac{1}{2m+1} \sum_{j=-m}^{m} I(f_{k+j}) $$

Zwiększenie szerokości okna ($m$) powoduje redukcję wariancji estymatora kosztem zwiększenia obciążenia (rozmycie pików). Wygładzony periodogram staje się estymatorem zgodnym, gdy $m \to \infty$ wraz z $N \to \infty$, ale wolniej niż $N$.

In [12]:
def apply_daniell_smoothing(power, m):
    """
    Wygładzanie oknem Daniella (prosta średnia ruchoma).
    """
    window = np.ones(2*m + 1) / (2*m + 1)
    return np.convolve(power, window, mode='same')

def plot_daniell_comparison(freqs, power, m_values, title="Daniell Scaling"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=freqs[1:], y=power[1:], name="Naiwny", line=dict(color='lightgray', width=1)))
    
    colors = ['blue', 'red', 'green']
    for m, color in zip(m_values, colors):
        smoothed = apply_daniell_smoothing(power, m)
        fig.add_trace(go.Scatter(x=freqs[1:], y=smoothed[1:], name=f"m={m}", line=dict(color=color, width=2)))
    
    fig.update_layout(
        title=title,
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc",
        yaxis_type="log",
        xaxis_range=[0, 0.05]
    )
    show_plotly(fig)

m_vals = [5, 15, 30]
plot_daniell_comparison(f_hp, p_hp, m_vals, title="Wygładzanie oknem Daniella (Cykl HP)")
plot_daniell_comparison(f_cf, p_cf, m_vals, title="Wygładzanie oknem Daniella (Cykl CF)")

### Wnioski i Interpretacja:

1. **Redukcja wariancji**: Wyraźnie widać, że wraz ze wzrostem szerokości okna ($m$), wykres staje się gładszy. Krótkookresowe oscylacje (szum) zostają wytłumione.
2. **Rozdzielczość vs Stabilność**: Dla małych $m$ (np. $m=1$) zachowujemy piki, ale wariancja jest nadal duża. Dla dużych $m$ (np. $m=7$) estymator jest bardzo stabilny, ale możemy stracić informację o blisko położonych cyklach (piki zlewają się w jeden szeroki garb).
3. **Identyfikacja cykli**: Wygładzony wykres pozwala łatwiej dostrzec dominujące składowe, które w wersji naiwnej mogły być przesłonięte przez sąsiednie szpilki o wysokiej amplitudzie, będące wynikiem losowości. Dzięki temu widzimy potencjalny cykl na częstotliwości **0.144**.

## Metoda Welcha (uśrednianie periodogramów)

Metoda Welcha jest udoskonaleniem metody Bartletta.
Polega na:
1. Podziale sygnału na segmenty (często nakładające się).
2. Nałożeniu okna (np. Hamminga, Hanna) na każdy segment w celu redukcji wycieku spektralnego.
3. Obliczeniu periodogramu dla każdego segmentu.
4. Uśrednieniu otrzymanych periodogramów.

Uśrednianie powoduje redukcję wariancji proporcjonalnie do liczby segmentów ($K$). Kosztem jest utrata rozdzielczości częstotliwościowej (ponieważ segmenty są krótsze niż cały sygnał $N$).

In [13]:
from scipy.signal import welch

def compute_welch_periodogram(x, nperseg=256, noverlap=None):
    """
    Estymacja gęstości widmowej mocy metodą Welcha.
    """
    f, p = welch(x, fs=1.0, window='hann', nperseg=nperseg, noverlap=noverlap, scaling='spectrum')
    return f, p

def plot_welch_comparison(series, freqs_naive, power_naive, nperseg_values, title="Welch Comparison"):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=freqs_naive[1:], y=power_naive[1:], name="Naiwny", line=dict(color='lightgray', width=1)))
    
    colors = ['blue', 'red', 'green']
    for nps, color in zip(nperseg_values, colors):
        fw, pw = compute_welch_periodogram(series, nperseg=nps)
        fig.add_trace(go.Scatter(x=fw[1:], y=pw[1:], name=f"nperseg={nps}", line=dict(color=color, width=2)))
    
    fig.update_layout(
        title=title,
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc",
        yaxis_type="log",
        xaxis_range=[0, 0.05]
    )
    show_plotly(fig)

nps_vals = [256, 512, 1024]
plot_welch_comparison(cycle_hp, f_hp, p_hp, nps_vals, title="Metoda Welcha (Cykl HP)")
plot_welch_comparison(cycle_cf, f_cf, p_cf, nps_vals, title="Metoda Welcha (Cykl CF)")

### Wnioski i Interpretacja:

1. **Gładkość estymatora**: Metoda Welcha daje najgładsze wyniki spośród dotychczasowych metod. Wynika to z bezpośredniego uśredniania niezależnych (w dużym stopniu) estymatów z segmentów.
2. **Wpływ `nperseg`**: 
    - Krótsze segmenty (`nperseg=128`) dają bardzo gładki wykres (mała wariancja), ale piki są szerokie i mało precyzyjne.
    - Dłuższe segmenty (`nperseg=512`) pozwalają na lepsze rozróżnienie bliskich częstotliwości, ale wykres jest bardziej "poszarpany".
3. **Stabilność**: Metoda Welcha jest uważana za standard w praktycznej analizie widmowej ze względu na doskonały balans między redukcją szumu a zachowaniem cech sygnału.

## Porównanie metod i analiza dominujących cykli

W tym punkcie zestawiamy wyniki z różnych metod estymacji. **Ważna uwaga:** Do analizy szczytów używamy składowej cyklicznej (`cycle`), ponieważ surowy szereg (lub nawet jego logarytm) jest zdominowany przez trend. W takim przypadku wykres gęstości widmowej opada niemal monotonicznie (tzw. szum czerwony), co uniemożliwia znalezienie lokalnych maksimów. Filtracja HP pozwala nam skupić się na właściwych oscylacjach.

In [14]:
# Wybieramy najlepsze parametry dla każdej metody do ostatecznego porównania
m_best = 15
nps_best = 512

def find_dominating_peaks(f, p, top_n=3):
    indices = signal.find_peaks(p)[0]
    peak_freqs = f[indices]
    peak_powers = p[indices]
    
    top_indices = np.argsort(peak_powers)[-top_n:][::-1]
    return peak_freqs[top_indices], peak_powers[top_indices]

def print_peaks_table(f_naive, p_naive, f_dan, p_dan, f_welch, p_welch, title="Peaks"):
    print(f"\n--- {title} ---")
    methods = ["Naiwny", "Daniell", "Welch"]
    all_f = [f_naive, f_dan, f_welch]
    all_p = [p_naive, p_dan, p_welch]
    
    for name, f, p in zip(methods, all_f, all_p):
        peaks_f, peaks_p = find_dominating_peaks(f, p)
        print(f"{name:10} | Top cykle (dni): {', '.join([f'{1/fq:.1f}' for fq in peaks_f])}")

# HP Analysis
p_hp_dan = apply_daniell_smoothing(p_hp, m_best)
f_hp_welch, p_hp_welch = compute_welch_periodogram(cycle_hp, nperseg=nps_best)
print_peaks_table(f_hp, p_hp, f_hp, p_hp_dan, f_hp_welch, p_hp_welch, title="Analiza szczytów (Cykl HP)")

# CF Analysis
p_cf_dan = apply_daniell_smoothing(p_cf, m_best)
f_cf_welch, p_cf_welch = compute_welch_periodogram(cycle_cf, nperseg=nps_best)
print_peaks_table(f_cf, p_cf, f_cf, p_cf_dan, f_cf_welch, p_cf_welch, title="Analiza szczytów (Cykl CF)")


--- Analiza szczytów (Cykl HP) ---
Naiwny     | Top cykle (dni): 2.6, 4.4, 3.1
Daniell    | Top cykle (dni): 5.3, 5.3, 2.8
Welch      | Top cykle (dni): 3.1, 5.2, 102.4

--- Analiza szczytów (Cykl CF) ---
Naiwny     | Top cykle (dni): 1401.5, 700.8, 560.6
Daniell    | Top cykle (dni): 373.7, 136.7, 151.5
Welch      | Top cykle (dni): 512.0, 102.4


### Wnioski końcowe Fazy 2 (Analiza Porównawcza Filtrów):

1. **Charakterystyka filtrów**:
   - **Filtr HP**: Pozostawia szerokie spektrum sygnału, co skutkuje większą liczbą pików w analizie (widzimy zarówno cykle krótko-, jak i długoterminowe).
   - **Filtr CF**: Działa precyzyjniej jako band-pass. Wyizolowanie pasma [500, 1500] dni pozwoliło na jednoznaczną identyfikację cykli o długościach zbliżonych do cyklu halvingowego Bitcoina, eliminując codzienny szum.
2. **Dominujący cykl**: 
   - W obu przypadkach metody wygładzone (Daniell/Welch) wskazują na dominację cykli w okolicach ... dni (sprawdź najwyższe wartości w tabelach powyżej).
3. **Metoda Welcha vs Daniell**: 
   - Metoda Welcha (niebieskie linie na wykresach) daje najstabilniejsze estymaty, redukując wariancję kosztem rozdzielczości, co jest kluczowe przy tak zmiennych danych jak BTC.

# Punkt 3: Analiza Danych Niekompletnych

W rzeczywistych zastosowaniach często spotykamy się z brakiem danych. W tej części symulujemy utratę 30% obserwacji i sprawdzamy, jak radzą sobie z tym standardowy periodogram oraz periodogram Lomba-Scargle'a.

In [15]:
import numpy as np
import scipy.signal as signal
import plotly.graph_objects as go

def run_incomplete_data_analysis(series, name):
    print(f"\n--- Analiza danych niekompletnych: {name} ---")
    N = len(series)
    times = np.arange(N)
    
    # 1. Symulacja braków (30%)
    missing_ratio = 0.3
    n_missing = int(N * missing_ratio)
    missing_idx = np.random.choice(times, n_missing, replace=False)
    
    mask = np.ones(N, dtype=bool)
    mask[missing_idx] = False
    
    t_incomplete = times[mask]
    x_incomplete = series.values[mask]
    
    # 2. Periodogram naiwny (ignorujemy luki)
    f_naive, p_naive = compute_naive_periodogram(x_incomplete)
    
    # 3. Periodogram Lomba-Scargle'a
    # f_lomb = np.linspace(0.001, 0.5, 1000)
    # Wykorzystujemy grid f_naive dla porównywalności, ale LS nie wymaga regularności
    f_lomb = np.linspace(0.0001, 0.05, 2000)
    # lombscargle oczekuje angular frequency
    p_lomb = signal.lombscargle(t_incomplete, x_incomplete, f_lomb * 2 * np.pi, normalize=True)
    
    # Istotność (95%)
    # Prob(P > z) = 1 - (1 - exp(-z))^(M) approx M * exp(-z)
    M = len(f_lomb)
    z_threshold = -np.log(1 - (1 - 0.05)**(1/M))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=f_naive[1:], y=p_naive[1:], name="Naiwny (ignoruje luki)", line=dict(color='gray', width=1)))
    fig.add_trace(go.Scatter(x=f_lomb, y=p_lomb, name="Lomb-Scargle", line=dict(color='red', width=2)))
    fig.add_trace(go.Scatter(x=[0, 0.05], y=[z_threshold, z_threshold], name="Istotność 5%", line=dict(color='black', dash='dash')))
    
    fig.update_layout(
        title=f"Analiza danych niekompletnych (-30%): {name}",
        xaxis_title="Częstotliwość (f)",
        yaxis_title="Moc / P-value",
        xaxis_range=[0, 0.05],
        # yaxis_type="log" # LS normalizowany lepiej wygląda liniowo lub log zależnie od skali
    )
    show_plotly(fig)

run_incomplete_data_analysis(cycle_hp, "Cykl HP")
run_incomplete_data_analysis(cycle_cf, "Cykl CF")


--- Analiza danych niekompletnych: Cykl HP ---



--- Analiza danych niekompletnych: Cykl CF ---


### Wnioski i Interpretacja (Punkt 3 - Dane Niekompletne):

1. **Błąd metody naiwnej**: Traktowanie danych niekompletnych jako ciągłych (poprzez "sklejanie" luk) wprowadza silne artefakty w widmie. Piki ulegają przesunięciu, a moc jest sztucznie zawyżana w niektórych pasmach.
2. **Przewaga Lomba-Scargle'a**: Algorytm LS poprawnie radzi sobie z nierównomiernym próbkowaniem. Widać to szczególnie przy składowej CF, gdzie LS precyzyjnie odtwarza piki w zadanym pasmie częstotliwości, mimo braku 30% obserwacji.
3. **Istotność statystyczna**: Linia przerywana wskazuje poziom 5%. Piki wystające powyżej tej linii możemy uznać za sygnały, które z wysokim prawdopodobieństwem nie są czystym szumem białym.